### Example 4.2.1.1: 1-D interpolation on equally spaced input points

In [1]:
#Setting Peridynamic parameters
class PDoperator_:
    def __init__(self, number_input_points = None, xmin = None, xmax = None, 
               number_output_points = None, xvec_org = None, yvec_org = None, 
               n1order = None, nsize = None, order = None):
        self.totnode_org = number_input_points
        self.totnode = number_output_points
        self.xmin = xmin
        self.xmax = xmax
        self.xvec_org = xvec_org 
        self.yvec_org = yvec_org
        self.n1order = n1order
        self.nsize = nsize
        self.order = order
    

In [2]:
#Function that generates grid
class GridGenerator:
    def __init__(init, PDoperator):
        
        xmin = PDoperator.xmin 
        xmax = PDoperator.xmax
        totnode = PDoperator.totnode
        
        xvec = []
        dvolume = []
        dx = (xmax - xmin)/ totnode
        for i in range(totnode):
            xvec.append(round(xmin + dx*i +dx/2.0,6))
            dvolume.append(round(dx*1*1,6))
        init.xvec = xvec
        init.dvolume = dvolume
    
    

In [3]:
import math
#In this class we create a pandas dataframe that hold the values for Xsi and their magnitudes
class FormXsiDataFrame:
    def __init__(init, xvec_org, xvec):
        xsivec = pd.DataFrame({'node':xvec_org})
        xsivec_mags = pd.DataFrame({'node':xvec_org})
    
        for i in range(len(xvec)):
            aux = []
            aux2 = []
            for j in range(len(xvec_org)):
                aux.append(xvec_org[j]-xvec[i])
                aux2.append(math.sqrt(((xvec_org[j]-xvec[i])**2)))
            xsivec[str(xvec[i])] = aux
            xsivec_mags[str(xvec[i])] = aux2
        
        init.xsivec = xsivec
        init.xsivec_mags = xsivec_mags


In [4]:
#This class creates a list of all p lists
class FormPlistPrism:
    def __init__(init, xsivec_matrix):
        totnode_org, totnode =  xsivec_matrix.shape
        p_list = np.zeros((totnode_org, totnode, 3)) #The 3 is hard coded which should be change later
        node_count = 0
        for node in xsivec_matrix:
            xsivec = xsivec_matrix[str(node)]
            xsi_count = 0
            for xsi in xsivec:
                aux = np.zeros((3,1))
                for j in range(3):
                    aux[j] = xsi**j                
                p_list[xsi_count, node_count, :] = aux.reshape((1, 3))
                xsi_count = xsi_count + 1
            node_count = node_count + 1
        
        init.p_list = p_list


In [5]:
import pandas as pd
#Function that finds the node family for each Peridynamic node
class GenerateNodeFamily:
    def __init__(init, xsi_df):
    
        xsivec_mags = xsi_df.xsivec_mags.copy()
        xsi_order = xsi_df.xsivec_mags.copy()
   
        xsi_order.set_index('node', inplace=True)    
        for i in xsi_order:
            xsi_order[str(i)] = xsi_order[str(i)].sort_values().keys()
        
        xsi_order_2 = pd.DataFrame({'node':xvec_org})
        for i in xsi_order:
            xsi_order_2[str(i)] = list(xsi_order[str(i)].sort_values().keys())
    
        #Calculating deltas (horizon) we are selecting the largest bond
        deltax = []    
        xsivec_mags = xsivec_mags.drop(columns="node")
        for i in xsivec_mags:
            deltax.append(max(list(xsivec_mags[str(i)])))
    
        init.xsi_order = xsi_order_2.drop(columns="node")
        init.deltax = deltax


In [6]:
#Function that creates lump volumes
class LumpVolumes:
    def __init__(init, totnode_org, totnode, xvec_org, xvec, dvolume):           
    
        dvolumes_org = [0] * totnode_org
        tiny = 1E-7
        rho = [0]*totnode_org
        
        for i in range(totnode):
            rhosum = 0.0
            for j in range(totnode_org):
                rho[j] = 1.0/((abs(xvec[i] - xvec_org[j]))**3 + tiny)
                rhosum = rhosum + rho[j]
            for j in range(totnode_org):
                dvolumes_org[j] = dvolumes_org[j] + (rho[j] / rhosum)*dvolume[i]
        rhosum = 0.0;
        for j in range(totnode_org):
            rhosum += dvolumes_org[j]
        print("Total volume of output points         = ",totnode*dvolume[0] )
        print("Total volume of computed input points = ", rhosum)
        
        init.dvolumes_org = dvolumes_org

In [7]:
#Function that calculates the Peridynamics weights
class FormWeightDataFrame:
    def __init__(init, xsivec, deltax, xvec):
        tiny = 0.000001
        weights = pd.DataFrame() 
        k=0    
        for i in xvec:
            xsivec_column = xsivec[str(i)].to_numpy()
            aux = []
            for xsi in xsivec_column:
                aux.append((deltax[k]/(xsi+tiny))**2)
            weights[str(i)] = aux
            k = k + 1  
        init.weights = weights

In [8]:
import math

class FormDiffAmat1D:
    def __init__(init, p_list, dvolumes_org, weights):
        node_mem = 0;
        #print(np.matmul(p_list[0][0],p_list[0][0]))
        #print(p_list[0][0].reshape((1, 3)))
        #print(np.ndarray.transpose(p_list[0][0].reshape((1, 3))))
        
        #print(np.matmul(np.transpose(p_list[0][0].reshape((1, 3))),p_list[0][0].reshape((1, 3))))
        aux = weights['0.0225']
        #print(p_list[0][0])
        #print(aux[0])
        #print(dvolumes_org[0])
        aux2 = 0
        aux2 = np.matmul(np.transpose(p_list[0][0].reshape((1, 3))),p_list[0][0].reshape((1, 3)))*aux[0]*dvolumes_org[0]
        +np.matmul(np.transpose(p_list[1][0].reshape((1, 3))),p_list[1][0].reshape((1, 3)))*aux[1]*dvolumes_org[1]
        +np.matmul(np.transpose(p_list[2][0].reshape((1, 3))),p_list[2][0].reshape((1, 3)))*aux[2]*dvolumes_org[2]
        +np.matmul(np.transpose(p_list[3][0].reshape((1, 3))),p_list[3][0].reshape((1, 3)))*aux[3]*dvolumes_org[3]
        +np.matmul(np.transpose(p_list[4][0].reshape((1, 3))),p_list[4][0].reshape((1, 3)))*aux[4]*dvolumes_org[4]
        +np.matmul(np.transpose(p_list[5][0].reshape((1, 3))),p_list[5][0].reshape((1, 3)))*aux[5]*dvolumes_org[5]
        +np.matmul(np.transpose(p_list[6][0].reshape((1, 3))),p_list[6][0].reshape((1, 3)))*aux[6]*dvolumes_org[6]
        +np.matmul(np.transpose(p_list[7][0].reshape((1, 3))),p_list[7][0].reshape((1, 3)))*aux[7]*dvolumes_org[7]
        +np.matmul(np.transpose(p_list[8][0].reshape((1, 3))),p_list[8][0].reshape((1, 3)))*aux[8]*dvolumes_org[8]
        +np.matmul(np.transpose(p_list[9][0].reshape((1, 3))),p_list[9][0].reshape((1, 3)))*aux[9]*dvolumes_org[9]     
        print(aux2)
        #print(np.matmul(np.transpose(p_list[0][0].reshape((1, 3),p_list[0][0].reshape((1, 3))))))
              #*dvolumes_org[node_mem]*weights[node_mem])
        #for p in plist:
        #    aux2 +=np.matmul(p,np.transpose(p))*dvolumes_org[node_mem]*weights[node_mem]
        #    print(aux2)
        #    node_mem = node_mem + 1
        #for i in x0_array:
        #    aux = np.zeros((nsize,1))
        #    for j in range(nsize):
        #        aux[j] = i**j
        #        print(aux[j])
          
            #aux2 +=np.matmul(aux,np.transpose(aux))
            #aux2 = aux2*dvolumes_org[node_mem]
            #aux2 +=np.matmul(aux,np.transpose(aux))*dvolumes_org[node_mem]*weights[node_mem]
            #aux2 +=np.matmul(aux,np.transpose(aux))
            #node_mem = node_mem + 1

In [9]:
import numpy as np
number_input_points = 10
xmin = 0
xmax = 9
number_output_points = 200
xvec_org = list(range(xmin,xmax+1))
yvec_org = [2, 2, 2, 2, 2, 3.2, 3.2, 3.2, 3.2, 3.2]
n1order = 2

PDoperator = PDoperator_(number_input_points, xmin, xmax, number_output_points, 
                        xvec_org, yvec_org, n1order)

#Generating the grid
gridgenerator = GridGenerator(PDoperator)

#Create xsi dataframe
xsi_df = FormXsiDataFrame(PDoperator.xvec_org, gridgenerator.xvec)

#Create plist

p_list = FormPlistPrism(xsi_df.xsivec.drop(columns="node"))


#Generate node family
NodeFam = GenerateNodeFamily(xsi_df)

#Create lump volumes
dvolumes_org = LumpVolumes(PDoperator.totnode_org, PDoperator.totnode, PDoperator.xvec_org, gridgenerator.xvec, 
                           gridgenerator.dvolume)

#Calculating Peridynamic weights
weights = FormWeightDataFrame(xsi_df.xsivec_mags.drop(columns="node"), NodeFam.deltax, gridgenerator.xvec)

#Form A array of A Matrices and p_vec_prism
DiffAmat1D = FormDiffAmat1D(p_list.p_list, dvolumes_org.dvolumes_org, weights.weights);

#print(xsi_df.xsivec)
#print(weights.weights.shape)
#print(p_list.p_list.shape)
#print(dvolumes_org.dvolume_org)



Total volume of output points         =  9.0
Total volume of computed input points =  9.0
[[ 8.04513282e+04 -1.81015488e+03  4.07284849e+01]
 [-1.81015488e+03  4.07284849e+01 -9.16390911e-01]
 [ 4.07284849e+01 -9.16390911e-01  2.06187955e-02]]


In [10]:
print(weights.weights)

          0.0225        0.0675       0.1125       0.1575       0.2025  \
0  159186.849743  17511.592245  6240.889050  3151.980383  1887.401112   
1      84.348314     91.758739   100.281663   110.156287   121.690333   
2      20.609972     21.365182    22.170978    23.032160    23.954123   
3       9.090902      9.278328     9.473598     9.677181     9.889585   
4       5.094367      5.159509     5.226587     5.295685     5.366891   
5       3.253030      3.279531     3.306629     3.334346     3.362702   
6       2.255649      2.267099     2.278753     2.290617     2.302697   
7       1.655431      1.660222     1.665083     1.670015     1.675020   
8       1.266418      1.268019     1.269639     1.271279     1.272939   
9       1.000000      1.000000     1.000000     1.000000     1.000000   

        0.2475      0.2925      0.3375      0.3825      0.4275  ...  \
0  1250.576671  886.201041  658.773874  507.571740  402.106152  ...   
1   135.285134  151.472171  170.967444  194.754389  22